In [1]:
import pyspark 
from sklearn.datasets import load_boston
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructField, StructType , DoubleType
from pyspark.sql.functions import corr

## config the spark

In [2]:
spark = SparkSession.builder.appName("BostonHousePricingSpark").master("local[*]").getOrCreate()

23/02/19 22:11:02 WARN Utils: Your hostname, MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
23/02/19 22:11:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/19 22:11:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/19 22:11:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


 ## load the boston dataset

In [3]:
boston = load_boston()

/Volumes/macbook/Anaconda/anaconda3/envs/py/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the 

## convert sklearn dataset to spark dataframe and save the spark dataframe to local

In [4]:
fields = [StructField(boston.feature_names[i],DoubleType(),True) for i in range(boston.data.shape[1])]
fields.append(StructField("Target",DoubleType(),True))
schema = StructType(fields)
print(schema)
data_list = boston.data.tolist()
target_list = boston.target.tolist()
data_target_list = [(data_list[i] + [target_list[i]]) for i in range(len(target_list))]
data = spark.createDataFrame(data_target_list,schema=schema)
data.show(2)

StructType(List(StructField(CRIM,DoubleType,true),StructField(ZN,DoubleType,true),StructField(INDUS,DoubleType,true),StructField(CHAS,DoubleType,true),StructField(NOX,DoubleType,true),StructField(RM,DoubleType,true),StructField(AGE,DoubleType,true),StructField(DIS,DoubleType,true),StructField(RAD,DoubleType,true),StructField(TAX,DoubleType,true),StructField(PTRATIO,DoubleType,true),StructField(B,DoubleType,true),StructField(LSTAT,DoubleType,true),StructField(Target,DoubleType,true)))


+-------+----+-----+----+-----+-----+----+------+---+-----+-------+-----+-----+------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|  TAX|PTRATIO|    B|LSTAT|Target|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+-----+-----+------+
|0.00632|18.0| 2.31| 0.0|0.538|6.575|65.2|  4.09|1.0|296.0|   15.3|396.9| 4.98|  24.0|
|0.02731| 0.0| 7.07| 0.0|0.469|6.421|78.9|4.9671|2.0|242.0|   17.8|396.9| 9.14|  21.6|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+-----+-----+------+
only showing top 2 rows



In [5]:
#save the Data
data.write.format("parquet").method('overwrite').option('header','true').save('data/raw/bostonHousing')

AnalysisException: path file:/Users/gaurav/Desktop/Datalaptop/Desktop/DataScience/gitProjects/Data-Science-Portfolio/BostonHousePricing/data/raw/bostonHousing already exists.

In [6]:
data.describe().toPandas()

,summary,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Target
0,count,506,506,506,506,506,506,506,506,506,506,506,506,506,506
1,mean,3.613523557312254,11.363636363636363,11.136778656126474,0.0691699604743083,0.5546950592885374,6.284634387351778,68.57490118577076,3.7950426877470353,9.549407114624506,408.2371541501976,18.45553359683795,356.67403162055354,12.653063241106722,22.53280632411067
2,stddev,8.601545105332487,23.32245299451514,6.860352940897584,0.25399404134041037,0.11587767566755591,0.7026171434153231,28.148861406903624,2.1057101266276113,8.707259384239368,168.53711605495903,2.164945523714439,91.29486438415786,7.14106151134857,9.197104087379817
3,min,0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73,5.0
4,max,88.9762,100.0,27.74,1.0,0.871,8.78,100.0,12.1265,24.0,711.0,22.0,396.9,37.97,50.0


## Exploratory Data Analysis 

In [ ]:
correlation  = []
colmns = data.columns
for i,col1 in enumerate(colmns):
    row = []
    for j,col2 in enumerate(colmns):
            corr_val = data.agg(corr(col1,col2)).first()[0]
            row.append(corr_val)
    correlation.append(row)
print(correlation)


In [ ]:
spark.createDataFrame(correlation,schema=schema).ToPandas()

## Independent feature or Dependent feature

In [ ]:
data.columns

In [7]:
from pyspark.ml.feature import StandardScaler , VectorAssembler
assembler = VectorAssembler(inputCols=data.columns[:-1],outputCol='features')
dataAssembled = assembler.transform(data)

In [ ]:
from pyspark.ml.stat import Correlation
Correlation.corr(dataAssembled,'features','pearson')

In [39]:
trainData , testData = dataAssembled.randomSplit([0.7,0.3],seed=123)

In [40]:
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures')
scaleModel = scaler.fit(trainData)
trainDataScaled = scaleModel.transform(trainData)
testDataScaled = scaleModel.transform(testData)

23/02/07 14:06:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [47]:
trainDataScaled = trainDataScaled.drop('features').withColumnRenamed('scaledFeatures','features')

In [48]:
testDataScaled = testDataScaled.drop('features').withColumnRenamed('scaledFeatures','features')

In [42]:
X_train = trainDataScaled.select('scaledFeatures')
Y_train = trainDataScaled.select('Target')
X_test = testDataScaled.select('scaledFeatures')
Y_test = testDataScaled.select('Target')

In [49]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features',labelCol='Target')
linearRegModel = lr.fit(trainDataScaled)


23/02/08 01:32:15 WARN Instrumentation: [2d1b6678] regParam is zero, which might cause numerical instability and overfitting.
23/02/08 01:32:16 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/08 01:32:16 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/02/08 01:32:16 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/02/08 01:32:16 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [54]:
output = linearRegModel.transform(testDataScaled)

In [55]:
output.toPandas()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Target,features,prediction
0,0.01360,75.0,4.00,0.0,0.410,5.888,47.6,7.3197,3.0,469.0,21.1,396.90,14.80,18.9,"[0.0014497563454740606, 3.229646407984387, 0.5...",14.412631
1,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,"[0.002909106666763758, 0.0, 1.0466127444842817...",30.513756
2,0.02875,28.0,15.04,0.0,0.464,6.211,28.9,3.6659,4.0,270.0,18.2,396.33,6.21,25.0,"[0.0030647422744396503, 1.205734658980838, 2.2...",29.103762
3,0.03359,75.0,2.95,0.0,0.428,7.024,15.8,5.4011,3.0,252.0,18.3,395.62,1.98,34.9,"[0.0035806849738583605, 3.229646407984387, 0.4...",33.679658
4,0.03551,25.0,4.86,0.0,0.426,6.167,46.7,5.4007,4.0,281.0,19.0,390.64,7.51,22.9,"[0.0037853564579252864, 1.0765488026614625, 0....",25.131852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,24.39380,0.0,18.10,0.0,0.700,4.652,100.0,1.4672,24.0,666.0,20.2,396.90,28.28,10.5,"[2.6003725250165544, 0.0, 2.679447054478854, 0...",6.508512
142,24.80170,0.0,18.10,0.0,0.693,5.349,96.0,1.7028,24.0,666.0,20.2,396.90,19.77,8.3,"[2.643854555407648, 0.0, 2.679447054478854, 0....",13.338400
143,25.04610,0.0,18.10,0.0,0.693,5.987,100.0,1.5888,24.0,666.0,20.2,396.90,26.77,5.6,"[2.669907529733667, 0.0, 2.679447054478854, 0....",11.614510
144,25.94060,0.0,18.10,0.0,0.679,5.304,89.1,1.6475,24.0,666.0,20.2,127.36,26.64,10.4,"[2.765260989367972, 0.0, 2.679447054478854, 0....",6.577023


In [65]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName='rmse',labelCol='Target',predictionCol='prediction')
evaluator.evaluate(linearRegModel.transform(testDataScaled))

4.258927493745456

In [72]:
linearRegModel.write().overwrite().save('/Users/gaurav/Desktop/Datalaptop/Desktop/DataScience/gitProjects/Data-Science-Portfolio/BostonHousePricing/model')

In [69]:
!pwd

/Users/gaurav/Desktop/Datalaptop/Desktop/DataScience/gitProjects/Data-Science-Portfolio/BostonHousePricing


TypeError: cannot pickle '_thread.RLock' object